In [1]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path_u= "ssd_images_1_to_17.csv"
dff = pd.read_csv(csv_path_u)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr = []
main_labels = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in dff.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr.append(sequence_of_frames)
        main_labels.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr))
print("Number of label:", len(main_labels))

Number of sequences: 748
Number of label: 748


In [4]:
main_arr = np.array(main_arr)
main_labels = np.array(main_labels)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l1,l2
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping

frames = 12
height = 100
width = 96
channels = 3
input_shape = (frames, height, width, channels)
num_classes = 6

def create_3dcnn(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(Conv3D(64, (3, 3, 3), activation='relu', padding="same", kernel_regularizer=l2(0),input_shape=input_shape))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))  

    model.add(Conv3D(128, (3, 3, 3), activation='relu', padding="same",kernel_regularizer=l2(0)))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Conv3D(256, (3, 3, 3), activation='relu', padding="same",kernel_regularizer=l2(0)))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Conv3D(512, (3, 3, 3), activation='relu', padding="same",kernel_regularizer=l2(0)))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Conv3D(1024, (3, 3, 3), activation='relu', padding="same",kernel_regularizer=l2(0)))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))
    
    #odel.add(Dropout(0.3))
    
    # Flatten the output for fully connected layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(1024, activation='relu',kernel_regularizer=l2(0.001))) 
    #model.add(Dropout(0.5))  
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    #model.add(Dropout(0.6))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    
    model.add(Dense(num_classes, activation='softmax')) 
    
    
    return model

model = create_3dcnn(input_shape, num_classes)

l = LabelEncoder()
main_labels_encoded = l.fit_transform(main_labels)

# Define the number of splits for stratified cross-validation
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

class_weights = compute_class_weight('balanced', classes=np.unique(main_labels_encoded), y=main_labels_encoded)
class_weights = {i: class_weights[i] for i in range(num_classes)}

cv_scores = []

for train_index, val_index in skf.split(main_arr, main_labels_encoded):
    x_train, x_val = main_arr[train_index], main_arr[val_index]
    y_train, y_val = main_labels_encoded[train_index], main_labels_encoded[val_index]

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    model.fit(x_train, y_train, epochs=50, batch_size=12, verbose=1,validation_data=(x_val,y_val),class_weight=class_weights, callbacks=[early_stopping])

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_val, y_val)
    cv_scores.append(test_accuracy)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

model.save('3DCNN_model_stratefied_reg_748.h5')
print("The model saved successfully.")

Epoch 1/50
42/42 [==============================] - 695s 17s/step - loss: 19.1982 - accuracy: 0.1566 - val_loss: 3.5922 - val_accuracy: 0.1800
Epoch 2/50
42/42 [==============================] - 687s 16s/step - loss: 3.4948 - accuracy: 0.1506 - val_loss: 3.4013 - val_accuracy: 0.1560
Epoch 3/50
42/42 [==============================] - 692s 16s/step - loss: 3.3430 - accuracy: 0.1426 - val_loss: 3.2822 - val_accuracy: 0.0880
Epoch 4/50
42/42 [==============================] - 686s 16s/step - loss: 3.2377 - accuracy: 0.0924 - val_loss: 3.1892 - val_accuracy: 0.0880
Epoch 5/50
42/42 [==============================] - 685s 16s/step - loss: 3.1516 - accuracy: 0.0924 - val_loss: 3.1103 - val_accuracy: 0.0880
Epoch 6/50
42/42 [==============================] - 737s 18s/step - loss: 3.0781 - accuracy: 0.0924 - val_loss: 3.0427 - val_accuracy: 0.0880
Epoch 7/50
42/42 [==============================] - 686s 16s/step - loss: 3.0143 - accuracy: 0.0924 - val_loss: 2.9831 - val_accuracy: 0.0880
Epoch

In [1]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "ssd_images_19_to_24.csv"
df = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr_test = []
main_labels_test = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in df.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr_test.append(sequence_of_frames)
        main_labels_test.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr_test))
print("Number of label:", len(main_labels_test))

Number of sequences: 264
Number of label: 264


In [2]:
main_arr_test=np.array(main_arr_test)
main_labels_test=np.array(main_labels_test)

In [4]:

# Make predictions
from keras.models import load_model
model=load_model('3DCNN_model_stratefied_simple_748.h5')

predictions = model.predict(main_arr_test)

print("The predictions are:", predictions)

The predictions are: [[0.74843234 0.00717864 0.13975252 0.00802646 0.01036539 0.08624456]
 [0.77950954 0.00156497 0.16474593 0.00383554 0.00271608 0.04762793]
 [0.41486663 0.03185083 0.27474058 0.01521947 0.06359416 0.19972833]
 ...
 [0.0824692  0.305121   0.12202128 0.02562871 0.0376549  0.4271049 ]
 [0.2798407  0.06024322 0.07164983 0.02791603 0.0294737  0.5308765 ]
 [0.09997299 0.10254887 0.15725988 0.01478636 0.03380863 0.59162325]]


In [5]:
predictions=np.array(predictions)

In [6]:
y_pred=[]
for i in predictions:
    print(i)
    y_pred.append(np.argmax(i))

[0.74843234 0.00717864 0.13975252 0.00802646 0.01036539 0.08624456]
[0.77950954 0.00156497 0.16474593 0.00383554 0.00271608 0.04762793]
[0.41486663 0.03185083 0.27474058 0.01521947 0.06359416 0.19972833]
[0.6375141  0.01477941 0.1127237  0.01999121 0.04737721 0.16761431]
[0.8224624  0.00362432 0.10369333 0.02589905 0.00272943 0.04159149]
[0.902324   0.0013502  0.05551793 0.00234329 0.00205785 0.0364067 ]
[0.02980064 0.06719185 0.03411928 0.01027106 0.01232563 0.8462915 ]
[0.5516454  0.03099353 0.18151659 0.02152723 0.0309214  0.18339582]
[0.02077698 0.64886004 0.03778726 0.21634069 0.00553883 0.07069625]
[0.38942245 0.04004279 0.3360835  0.13183461 0.01244267 0.09017389]
[0.00151842 0.94909346 0.00418695 0.00574947 0.00467889 0.03477262]
[0.03392508 0.26145008 0.01664246 0.65642786 0.00258717 0.02896734]
[0.07072014 0.16736577 0.29903707 0.2189822  0.10616378 0.13773099]
[0.320115   0.00933296 0.5296939  0.08669178 0.01155822 0.04260818]
[0.28339213 0.0794808  0.11480283 0.3677421  0.0

In [7]:
y_pred=np.array(y_pred)
len(y_pred)

264

In [8]:
from sklearn.metrics import accuracy_score
test_accuracy=accuracy_score(main_labels_test,y_pred)
#print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Test Accuracy: 0.4128787878787879


In [9]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(main_labels_test,y_pred)
print(cm)

[[25  5  3  9  5  1]
 [ 1 34  3  4  1  5]
 [13  9  9  8  6  3]
 [ 4 23  1 20  0  0]
 [ 1  2  4  0 13  4]
 [10  8  7  4 11  8]]
